In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn import tree

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TravelInsurancePrediction.csv')
df.head()

FileNotFoundError: ignored

In [ ]:
df_E = pd.get_dummies(df['Employment Type'], prefix = 'Employment Type')

NameError: ignored

In [ ]:
df_G = pd.get_dummies(df['GraduateOrNot'], prefix = 'GraduateOrNot')

In [ ]:
df_C =pd.get_dummies(df['ChronicDiseases'], prefix = 'ChronicDiseases')

In [ ]:
df_F = pd.get_dummies(df['FrequentFlyer'], prefix = 'FrequentFlyer')

In [ ]:
df_Ev = pd.get_dummies(df['EverTravelledAbroad'], prefix = 'EverTravelledAbroad')

In [ ]:
df1 = pd.concat([df, df_E, df_G, df_C, df_F, df_Ev], axis=1)

In [ ]:
df1 = df1[['Age','AnnualIncome','FamilyMembers','Employment Type_Government Sector','Employment Type_Private Sector/Self Employed','GraduateOrNot_No','GraduateOrNot_Yes','ChronicDiseases_0','ChronicDiseases_1','FrequentFlyer_No','FrequentFlyer_Yes','EverTravelledAbroad_No','EverTravelledAbroad_Yes','TravelInsurance']]

In [ ]:
df1.head()

,Age,AnnualIncome,FamilyMembers,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,GraduateOrNot_No,GraduateOrNot_Yes,ChronicDiseases_0,ChronicDiseases_1,FrequentFlyer_No,FrequentFlyer_Yes,EverTravelledAbroad_No,EverTravelledAbroad_Yes,TravelInsurance
0,31,400000,6,1,0,0,1,0,1,1,0,1,0,0
1,31,1250000,7,0,1,0,1,1,0,1,0,1,0,0
2,34,500000,4,0,1,0,1,0,1,1,0,1,0,1
3,28,700000,3,0,1,0,1,0,1,1,0,1,0,0
4,28,700000,8,0,1,0,1,0,1,0,1,1,0,0


In [ ]:
from sklearn.model_selection import train_test_split

# 인자로 입력받은 DataFrame을 복사 한 뒤 Time 컬럼만 삭제하고 복사된 DataFrame 반환
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    return df_copy

In [ ]:
# 사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수.
def get_train_test_dataset(df=None):
    # 인자로 입력된 DataFrame의 사전 데이터 가공이 완료된 복사 DataFrame 반환
    df_copy = get_preprocessed_df(df)
    # DataFrame의 맨 마지막 컬럼이 레이블, 나머지는 피처들
    X_features = df_copy.iloc[:, :-1]
    y_target = df_copy.iloc[:, -1]
    # train_test_split( )으로 학습과 테스트 데이터 분할. stratify=y_target으로 층화추출 기반 균등분할
    X_train, X_test, y_train, y_test = \
    train_test_split(X_features, y_target, test_size=0.2, random_state=0, stratify=y_target)
    # 학습과 테스트 데이터 세트 반환
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_dataset(df1)

In [ ]:
print('학습 데이터 레이블 값 비율')
print(y_train.value_counts()/y_train.shape[0] * 100)
print('테스트 데이터 레이블 값 비율')
print(y_test.value_counts()/y_test.shape[0] * 100)

학습 데이터 레이블 값 비율
0    64.254248
1    35.745752
Name: TravelInsurance, dtype: float64
테스트 데이터 레이블 값 비율
0    64.321608
1    35.678392
Name: TravelInsurance, dtype: float64


In [ ]:
# 평가 함수
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [ ]:
import time
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    start_time = time.time()
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)
    print("수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=1000,n_jobs=-1)
get_model_train_eval(rf_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[231  25]
 [ 53  89]]
정확도: 0.8040, 정밀도: 0.7807, 재현율: 0.6268, F1: 0.6953
수행 시간: 3.0 초 


In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)
get_model_train_eval(lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[223  33]
 [ 53  89]]
정확도: 0.7839, 정밀도: 0.7295, 재현율: 0.6268, F1: 0.6742
수행 시간: 1.0 초 


In [ ]:
from sklearn import metrics

In [ ]:
rf_clf.fit(X_train, y_train)
y_hat = rf_clf.predict(X_test)
rf_matrix = metrics.confusion_matrix(y_test, y_hat)
print(rf_matrix)
rf_report = metrics.classification_report(y_test, y_hat)
print(rf_report)

[[229  27]
 [ 53  89]]
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       256
           1       0.77      0.63      0.69       142

    accuracy                           0.80       398
   macro avg       0.79      0.76      0.77       398
weighted avg       0.80      0.80      0.79       398



In [ ]:
lgbm_clf.fit(X_train, y_train)
y_hat = lgbm_clf.predict(X_test)
lgbm_matrix = metrics.confusion_matrix(y_test, y_hat)
print(lgbm_matrix)
lgbm_report = metrics.classification_report(y_test, y_hat)
print(lgbm_report)

[[223  33]
 [ 53  89]]
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       256
           1       0.73      0.63      0.67       142

    accuracy                           0.78       398
   macro avg       0.77      0.75      0.76       398
weighted avg       0.78      0.78      0.78       398



In [ ]:
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)

tree_model.fit(X_train, y_train)
y_hat = tree_model.predict(X_test)
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)
tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

[[253   3]
 [ 57  85]]
              precision    recall  f1-score   support

           0       0.82      0.99      0.89       256
           1       0.97      0.60      0.74       142

    accuracy                           0.85       398
   macro avg       0.89      0.79      0.82       398
weighted avg       0.87      0.85      0.84       398



In [ ]:
df2 = df1[['Age','AnnualIncome','FamilyMembers','Employment Type_Government Sector','Employment Type_Private Sector/Self Employed','GraduateOrNot_No','GraduateOrNot_Yes','FrequentFlyer_No','FrequentFlyer_Yes','EverTravelledAbroad_No','EverTravelledAbroad_Yes','TravelInsurance']]

In [ ]:
from sklearn.model_selection import train_test_split

# 인자로 입력받은 DataFrame을 복사 한 뒤 Time 컬럼만 삭제하고 복사된 DataFrame 반환
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    return df_copy

In [ ]:
# 사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수.
def get_train_test_dataset(df=None):
    # 인자로 입력된 DataFrame의 사전 데이터 가공이 완료된 복사 DataFrame 반환
    df_copy = get_preprocessed_df(df)
    # DataFrame의 맨 마지막 컬럼이 레이블, 나머지는 피처들
    X_features = df_copy.iloc[:, :-1]
    y_target = df_copy.iloc[:, -1]
    # train_test_split( )으로 학습과 테스트 데이터 분할. stratify=y_target으로 층화추출 기반 균등분할
    X_train, X_test, y_train, y_test = \
    train_test_split(X_features, y_target, test_size=0.2, random_state=0, stratify=y_target)
    # 학습과 테스트 데이터 세트 반환
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_dataset(df2)

In [ ]:
# 평가 함수
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [ ]:
import time
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    start_time = time.time()
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)
    print("수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)
get_model_train_eval(lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[224  32]
 [ 51  91]]
정확도: 0.7915, 정밀도: 0.7398, 재현율: 0.6408, F1: 0.6868
수행 시간: 1.0 초 


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=1000,n_jobs=-1)
get_model_train_eval(rf_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[233  23]
 [ 52  90]]
정확도: 0.8116, 정밀도: 0.7965, 재현율: 0.6338, F1: 0.7059
수행 시간: 2.8 초 


In [ ]:
rf_clf.fit(X_train, y_train)
y_hat = rf_clf.predict(X_test)
rf_matrix = metrics.confusion_matrix(y_test, y_hat)
print(rf_matrix)
rf_report = metrics.classification_report(y_test, y_hat)
print(rf_report)

[[233  23]
 [ 51  91]]
              precision    recall  f1-score   support

           0       0.82      0.91      0.86       256
           1       0.80      0.64      0.71       142

    accuracy                           0.81       398
   macro avg       0.81      0.78      0.79       398
weighted avg       0.81      0.81      0.81       398



In [ ]:
lgbm_clf.fit(X_train, y_train)
y_hat = lgbm_clf.predict(X_test)
lgbm_matrix = metrics.confusion_matrix(y_test, y_hat)
print(lgbm_matrix)
lgbm_report = metrics.classification_report(y_test, y_hat)
print(lgbm_report)

[[224  32]
 [ 51  91]]
              precision    recall  f1-score   support

           0       0.81      0.88      0.84       256
           1       0.74      0.64      0.69       142

    accuracy                           0.79       398
   macro avg       0.78      0.76      0.77       398
weighted avg       0.79      0.79      0.79       398



In [ ]:
rf_clf.fit(X_train, y_train)
y_hat = rf_clf.predict(X_test)
rf_matrix = metrics.confusion_matrix(y_test, y_hat)
print(rf_matrix)
rf_report = metrics.classification_report(y_test, y_hat)
print(rf_report)

[[234  22]
 [ 52  90]]
              precision    recall  f1-score   support

           0       0.82      0.91      0.86       256
           1       0.80      0.63      0.71       142

    accuracy                           0.81       398
   macro avg       0.81      0.77      0.79       398
weighted avg       0.81      0.81      0.81       398



In [ ]:
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)

tree_model.fit(X_train, y_train)
y_hat = tree_model.predict(X_test)
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)
tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

[[251   5]
 [ 57  85]]
              precision    recall  f1-score   support

           0       0.81      0.98      0.89       256
           1       0.94      0.60      0.73       142

    accuracy                           0.84       398
   macro avg       0.88      0.79      0.81       398
weighted avg       0.86      0.84      0.83       398



In [ ]:
df3 = df1[['Age','AnnualIncome','FamilyMembers','Employment Type_Government Sector','Employment Type_Private Sector/Self Employed','FrequentFlyer_No','FrequentFlyer_Yes','EverTravelledAbroad_No','EverTravelledAbroad_Yes','TravelInsurance']]

In [ ]:
from sklearn.model_selection import train_test_split

# 인자로 입력받은 DataFrame을 복사 한 뒤 Time 컬럼만 삭제하고 복사된 DataFrame 반환
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    return df_copy

In [ ]:
# 사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수.
def get_train_test_dataset(df=None):
    # 인자로 입력된 DataFrame의 사전 데이터 가공이 완료된 복사 DataFrame 반환
    df_copy = get_preprocessed_df(df)
    # DataFrame의 맨 마지막 컬럼이 레이블, 나머지는 피처들
    X_features = df_copy.iloc[:, :-1]
    y_target = df_copy.iloc[:, -1]
    # train_test_split( )으로 학습과 테스트 데이터 분할. stratify=y_target으로 층화추출 기반 균등분할
    X_train, X_test, y_train, y_test = \
    train_test_split(X_features, y_target, test_size=0.2, random_state=0, stratify=y_target)
    # 학습과 테스트 데이터 세트 반환
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_dataset(df3)

In [ ]:
# 평가 함수
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [ ]:
import time
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    start_time = time.time()
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)
    print("수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)
get_model_train_eval(lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[224  32]
 [ 50  92]]
정확도: 0.7940, 정밀도: 0.7419, 재현율: 0.6479, F1: 0.6917
수행 시간: 1.0 초 


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=1000,n_jobs=-1)
get_model_train_eval(rf_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[233  23]
 [ 51  91]]
정확도: 0.8141, 정밀도: 0.7982, 재현율: 0.6408, F1: 0.7109
수행 시간: 2.8 초 


In [ ]:
lgbm_clf.fit(X_train, y_train)
y_hat = lgbm_clf.predict(X_test)
lgbm_matrix = metrics.confusion_matrix(y_test, y_hat)
print(lgbm_matrix)
lgbm_report = metrics.classification_report(y_test, y_hat)
print(lgbm_report)

[[224  32]
 [ 50  92]]
              precision    recall  f1-score   support

           0       0.82      0.88      0.85       256
           1       0.74      0.65      0.69       142

    accuracy                           0.79       398
   macro avg       0.78      0.76      0.77       398
weighted avg       0.79      0.79      0.79       398



In [ ]:
rf_clf.fit(X_train, y_train)
y_hat = rf_clf.predict(X_test)
rf_matrix = metrics.confusion_matrix(y_test, y_hat)
print(rf_matrix)
rf_report = metrics.classification_report(y_test, y_hat)
print(rf_report)

[[234  22]
 [ 51  91]]
              precision    recall  f1-score   support

           0       0.82      0.91      0.87       256
           1       0.81      0.64      0.71       142

    accuracy                           0.82       398
   macro avg       0.81      0.78      0.79       398
weighted avg       0.82      0.82      0.81       398



In [ ]:
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)

tree_model.fit(X_train, y_train)
y_hat = tree_model.predict(X_test)
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)
tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

[[253   3]
 [ 57  85]]
              precision    recall  f1-score   support

           0       0.82      0.99      0.89       256
           1       0.97      0.60      0.74       142

    accuracy                           0.85       398
   macro avg       0.89      0.79      0.82       398
weighted avg       0.87      0.85      0.84       398



In [ ]:
df4 = df1[['Age','AnnualIncome','FamilyMembers','Employment Type_Government Sector','Employment Type_Private Sector/Self Employed','GraduateOrNot_No','GraduateOrNot_Yes','ChronicDiseases_0','ChronicDiseases_1','EverTravelledAbroad_No','EverTravelledAbroad_Yes','TravelInsurance']]

In [ ]:
from sklearn.model_selection import train_test_split

# 인자로 입력받은 DataFrame을 복사 한 뒤 Time 컬럼만 삭제하고 복사된 DataFrame 반환
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    return df_copy

In [ ]:
# 사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수.
def get_train_test_dataset(df=None):
    # 인자로 입력된 DataFrame의 사전 데이터 가공이 완료된 복사 DataFrame 반환
    df_copy = get_preprocessed_df(df)
    # DataFrame의 맨 마지막 컬럼이 레이블, 나머지는 피처들
    X_features = df_copy.iloc[:, :-1]
    y_target = df_copy.iloc[:, -1]
    # train_test_split( )으로 학습과 테스트 데이터 분할. stratify=y_target으로 층화추출 기반 균등분할
    X_train, X_test, y_train, y_test = \
    train_test_split(X_features, y_target, test_size=0.2, random_state=0, stratify=y_target)
    # 학습과 테스트 데이터 세트 반환
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_dataset(df4)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

undersample = RandomUnderSampler(sampling_strategy='majority')
X_train_under, y_train_under = undersample.fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# 평가 함수
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [ ]:
import time
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    start_time = time.time()
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)
    print("수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)
get_model_train_eval(lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[225  31]
 [ 52  90]]
정확도: 0.7915, 정밀도: 0.7438, 재현율: 0.6338, F1: 0.6844
수행 시간: 1.0 초 


In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)
get_model_train_eval(lgbm_clf, ftr_train=X_train_under, ftr_test=X_test, tgt_train=y_train_under, tgt_test=y_test)

오차 행렬
[[198  58]
 [ 41 101]]
정확도: 0.7513, 정밀도: 0.6352, 재현율: 0.7113, F1: 0.6711
수행 시간: 0.7 초 


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=1000,n_jobs=-1)
get_model_train_eval(rf_clf, ftr_train=X_train_under, ftr_test=X_test, tgt_train=y_train_under, tgt_test=y_test)

오차 행렬
[[203  53]
 [ 42 100]]
정확도: 0.7613, 정밀도: 0.6536, 재현율: 0.7042, F1: 0.6780
수행 시간: 2.8 초 


In [ ]:
lgbm_clf.fit(X_train_under, y_train_under)
y_hat = lgbm_clf.predict(X_test)
lgbm_matrix = metrics.confusion_matrix(y_test, y_hat)
print(lgbm_matrix)
lgbm_report = metrics.classification_report(y_test, y_hat)
print(lgbm_report)

[[198  58]
 [ 41 101]]
              precision    recall  f1-score   support

           0       0.83      0.77      0.80       256
           1       0.64      0.71      0.67       142

    accuracy                           0.75       398
   macro avg       0.73      0.74      0.74       398
weighted avg       0.76      0.75      0.75       398



In [ ]:
rf_clf.fit(X_train_under, y_train_under)
y_hat = rf_clf.predict(X_test)
rf_matrix = metrics.confusion_matrix(y_test, y_hat)
print(rf_matrix)
rf_report = metrics.classification_report(y_test, y_hat)
print(rf_report)

[[198  58]
 [ 42 100]]
              precision    recall  f1-score   support

           0       0.82      0.77      0.80       256
           1       0.63      0.70      0.67       142

    accuracy                           0.75       398
   macro avg       0.73      0.74      0.73       398
weighted avg       0.76      0.75      0.75       398



In [ ]:
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)

tree_model.fit(X_train_under, y_train_under)
y_hat = tree_model.predict(X_test)
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)
tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

[[253   3]
 [ 57  85]]
              precision    recall  f1-score   support

           0       0.82      0.99      0.89       256
           1       0.97      0.60      0.74       142

    accuracy                           0.85       398
   macro avg       0.89      0.79      0.82       398
weighted avg       0.87      0.85      0.84       398



In [ ]:
df5 = df1[['Age','AnnualIncome','FamilyMembers','GraduateOrNot_No','GraduateOrNot_Yes','ChronicDiseases_0','ChronicDiseases_1','FrequentFlyer_No','FrequentFlyer_Yes','EverTravelledAbroad_No','EverTravelledAbroad_Yes','TravelInsurance']]

In [ ]:
from sklearn.model_selection import train_test_split

# 인자로 입력받은 DataFrame을 복사 한 뒤 Time 컬럼만 삭제하고 복사된 DataFrame 반환
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    return df_copy

In [ ]:
# 사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수.
def get_train_test_dataset(df=None):
    # 인자로 입력된 DataFrame의 사전 데이터 가공이 완료된 복사 DataFrame 반환
    df_copy = get_preprocessed_df(df)
    # DataFrame의 맨 마지막 컬럼이 레이블, 나머지는 피처들
    X_features = df_copy.iloc[:, :-1]
    y_target = df_copy.iloc[:, -1]
    # train_test_split( )으로 학습과 테스트 데이터 분할. stratify=y_target으로 층화추출 기반 균등분할
    X_train, X_test, y_train, y_test = \
    train_test_split(X_features, y_target, test_size=0.2, random_state=0, stratify=y_target)
    # 학습과 테스트 데이터 세트 반환
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_dataset(df4)

In [ ]:
# 평가 함수
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [ ]:
import time
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    start_time = time.time()
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)
    print("수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)
get_model_train_eval(lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[225  31]
 [ 52  90]]
정확도: 0.7915, 정밀도: 0.7438, 재현율: 0.6338, F1: 0.6844
수행 시간: 1.0 초 


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=1000,n_jobs=-1)
get_model_train_eval(rf_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[237  19]
 [ 55  87]]
정확도: 0.8141, 정밀도: 0.8208, 재현율: 0.6127, F1: 0.7016
수행 시간: 2.8 초 


In [ ]:
lgbm_clf.fit(X_train, y_train)
y_hat = lgbm_clf.predict(X_test)
lgbm_matrix = metrics.confusion_matrix(y_test, y_hat)
print(lgbm_matrix)
lgbm_report = metrics.classification_report(y_test, y_hat)
print(lgbm_report)

[[225  31]
 [ 52  90]]
              precision    recall  f1-score   support

           0       0.81      0.88      0.84       256
           1       0.74      0.63      0.68       142

    accuracy                           0.79       398
   macro avg       0.78      0.76      0.76       398
weighted avg       0.79      0.79      0.79       398



In [ ]:
rf_clf.fit(X_train, y_train)
y_hat = rf_clf.predict(X_test)
rf_matrix = metrics.confusion_matrix(y_test, y_hat)
print(rf_matrix)
rf_report = metrics.classification_report(y_test, y_hat)
print(rf_report)

[[237  19]
 [ 54  88]]
              precision    recall  f1-score   support

           0       0.81      0.93      0.87       256
           1       0.82      0.62      0.71       142

    accuracy                           0.82       398
   macro avg       0.82      0.77      0.79       398
weighted avg       0.82      0.82      0.81       398



In [ ]:
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)

tree_model.fit(X_train, y_train)
y_hat = tree_model.predict(X_test)
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)
tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

[[253   3]
 [ 57  85]]
              precision    recall  f1-score   support

           0       0.82      0.99      0.89       256
           1       0.97      0.60      0.74       142

    accuracy                           0.85       398
   macro avg       0.89      0.79      0.82       398
weighted avg       0.87      0.85      0.84       398



In [ ]:
df6 = df1[['Age','AnnualIncome','FamilyMembers','Employment Type_Government Sector','Employment Type_Private Sector/Self Employed','GraduateOrNot_No','GraduateOrNot_Yes','ChronicDiseases_0','ChronicDiseases_1','FrequentFlyer_No','FrequentFlyer_Yes','TravelInsurance']]

In [ ]:
from sklearn.model_selection import train_test_split

# 인자로 입력받은 DataFrame을 복사 한 뒤 Time 컬럼만 삭제하고 복사된 DataFrame 반환
def get_preprocessed_df(df=None):
    df_copy = df.copy()
    return df_copy

In [ ]:
# 사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수.
def get_train_test_dataset(df=None):
    # 인자로 입력된 DataFrame의 사전 데이터 가공이 완료된 복사 DataFrame 반환
    df_copy = get_preprocessed_df(df)
    # DataFrame의 맨 마지막 컬럼이 레이블, 나머지는 피처들
    X_features = df_copy.iloc[:, :-1]
    y_target = df_copy.iloc[:, -1]
    # train_test_split( )으로 학습과 테스트 데이터 분할. stratify=y_target으로 층화추출 기반 균등분할
    X_train, X_test, y_train, y_test = \
    train_test_split(X_features, y_target, test_size=0.2, random_state=0, stratify=y_target)
    # 학습과 테스트 데이터 세트 반환
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_dataset(df6)

In [ ]:
# 평가 함수
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [ ]:
import time
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    start_time = time.time()
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)
    print("수행 시간: {0:.1f} 초 ".format(time.time() - start_time))

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average=False)
get_model_train_eval(lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[221  35]
 [ 50  92]]
정확도: 0.7864, 정밀도: 0.7244, 재현율: 0.6479, F1: 0.6840
수행 시간: 1.0 초 


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=1000,n_jobs=-1)
get_model_train_eval(rf_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

오차 행렬
[[224  32]
 [ 50  92]]
정확도: 0.7940, 정밀도: 0.7419, 재현율: 0.6479, F1: 0.6917
수행 시간: 2.9 초 


In [ ]:
lgbm_clf.fit(X_train, y_train)
y_hat = lgbm_clf.predict(X_test)
lgbm_matrix = metrics.confusion_matrix(y_test, y_hat)
print(lgbm_matrix)
lgbm_report = metrics.classification_report(y_test, y_hat)
print(lgbm_report)

[[221  35]
 [ 50  92]]
              precision    recall  f1-score   support

           0       0.82      0.86      0.84       256
           1       0.72      0.65      0.68       142

    accuracy                           0.79       398
   macro avg       0.77      0.76      0.76       398
weighted avg       0.78      0.79      0.78       398



In [ ]:
rf_clf.fit(X_train, y_train)
y_hat = rf_clf.predict(X_test)
rf_matrix = metrics.confusion_matrix(y_test, y_hat)
print(rf_matrix)
rf_report = metrics.classification_report(y_test, y_hat)
print(rf_report)

[[227  29]
 [ 50  92]]
              precision    recall  f1-score   support

           0       0.82      0.89      0.85       256
           1       0.76      0.65      0.70       142

    accuracy                           0.80       398
   macro avg       0.79      0.77      0.78       398
weighted avg       0.80      0.80      0.80       398



In [ ]:
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)

tree_model.fit(X_train, y_train)
y_hat = tree_model.predict(X_test)
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)
tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

[[253   3]
 [ 57  85]]
              precision    recall  f1-score   support

           0       0.82      0.99      0.89       256
           1       0.97      0.60      0.74       142

    accuracy                           0.85       398
   macro avg       0.89      0.79      0.82       398
weighted avg       0.87      0.85      0.84       398



In [ ]:
df7 = df1[['Age','AnnualIncome','FamilyMembers','Employment Type_Government Sector','Employment Type_Private Sector/Self Employed','FrequentFlyer_No','FrequentFlyer_Yes','EverTravelledAbroad_No','EverTravelledAbroad_Yes','TravelInsurance']]